In [1]:
%reload_ext autoreload
%autoreload 2
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style()

from datasets import load_dataset
from urllib.parse import urlparse

In [3]:
os.makedirs("data", exist_ok=True)

In [106]:
import requests
from bs4 import BeautifulSoup


def scrape_website(url, text_div_class=None, request_headers={}):
    try:
        response = requests.get(url, headers=request_headers)
        response.raise_for_status()  # Check if the request was successful

        soup = BeautifulSoup(response.content, 'html.parser')

        if not text_div_class:  # If no text_div_class is provided, return the whole page content
            return soup.get_text(strip=True, separator="\n") 

        content_div = soup.find('div', class_=text_div_class)
        if content_div:
            return content_div.get_text(strip=True, separator="\n")

        print(f"No div with class '{text_div_class}' found.") 
        return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

In [100]:
import random
import time
from tqdm import tqdm


# To avoid 403 Client Error: Forbidden for url
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
]

sleep_times = [1, 2, 3, 4, 5]

# 1. The Onion


## Source 1 (text available)

https://huggingface.co/datasets/Biddls/Onion_News

- The header and body of the news article is split by a ' #~# ' token
- Lines with just the token had no body or no header and can be skipped


In [4]:
onion_ds = load_dataset("Biddls/Onion_News")

In [5]:
onion_df = pd.DataFrame(onion_ds['train'])

In [6]:
onion_df.head()

,text
0,Relaxed Marie Kondo Now Says She Perfectly Hap...
1,U.S. Officials Call For Correct Amount Of Viol...
2,Kamala Harris Asks Communications Assistant If...
3,25 Arrested In Fake Nursing School Diploma Sch...
4,"World’s Oldest American Dies At 72 #~# OXNARD,..."


In [7]:
onion_df.text.values[0][:150]

'Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage #~# LOS ANGELES—Admitting that she’d made some major lifestyle changes si'

In [8]:
onion_df.text.str.strip().str.match("#~#").sum()

0

In [9]:
onion_df[["headline", "body"]] = onion_df["text"].str.split(" #~# ", expand=True)
onion_df.drop(columns=["text"], inplace=True)

onion_df.head()

,headline,body
0,Relaxed Marie Kondo Now Says She Perfectly Hap...,LOS ANGELES—Admitting that she’d made some maj...
1,U.S. Officials Call For Correct Amount Of Viol...,WASHINGTON—Addressing the need for swift and m...
2,Kamala Harris Asks Communications Assistant If...,WASHINGTON—In an effort to expand her professi...
3,25 Arrested In Fake Nursing School Diploma Scheme,Federal agents have arrested 25 suspects accus...
4,World’s Oldest American Dies At 72,"OXNARD, CA—Living longer than any citizen in t..."


In [10]:
len(onion_df)

33880

In [11]:
onion_df.to_csv("data/theonion.csv", index=False)

## Source 2 (need to scrape)

https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection/data

In [68]:
import json

with open("data/Sarcasm_Headlines_Dataset.json", "r") as f:
    sarcasm_ds = [json.loads(row) for row in f]

In [84]:
sarcasm_df = pd.DataFrame(sarcasm_ds)

sarcasm_df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [85]:
sarcasm_df = sarcasm_df.query("is_sarcastic == 1").drop(columns=["is_sarcastic"])

len(sarcasm_df)

11724

In [86]:
sarcasm_df = sarcasm_df.rename(columns={"article_link": "url"})

sarcasm_df.head()

,url,headline
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up..."
8,https://politics.theonion.com/top-snake-handle...,top snake handler leaves sinking huckabee camp...
15,https://entertainment.theonion.com/nuclear-bom...,nuclear bomb detonates during rehearsal for 's...
16,https://www.theonion.com/cosby-lawyer-asks-why...,cosby lawyer asks why accusers didn't come for...


In [87]:
sarcasm_df.url.values[0]

'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697'

In [88]:
sarcasm_df.url.values[4]

'https://www.theonion.com/cosby-lawyer-asks-why-accusers-didn-t-come-forward-to-b-1819577265'

In [89]:
onion_scrape_test = scrape_website(sarcasm_df.url.values[0], 'entry-content single-post-content wp-block-post-content has-echo-font-size is-layout-flow wp-block-post-content-is-layout-flow')

print(onion_scrape_test)

Error fetching https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697: HTTPSConnectionPool(host='local.theonion.com', port=443): Max retries exceeded with url: /mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'local.theonion.com'. (_ssl.c:992)")))
None


In [90]:
onion_scrape_test = scrape_website(sarcasm_df.url.values[4], 'entry-content single-post-content wp-block-post-content has-echo-font-size is-layout-flow wp-block-post-content-is-layout-flow')

print(onion_scrape_test)

LOS ANGELES—Responding to recent allegations that his client sexually assaulted as many as 20 women over the course of his career, an attorney for comedian Bill Cosby asked Wednesday why none of the alleged victims came forward and allowed themselves to be smeared by the entertainer’s powerful attorneys years ago. “One has to wonder why all these claims are being made now, when these women very easily could have spoken up 40 years ago and had my client’s legal team ruin them,” said Cosby lawyer Martin Singer, suggesting that if the assertions were indeed true, the purported victims have had more than ample time to let Cosby’s team of high-priced legal advisors assassinate their character and threaten them with countersuits for defamation. “It’s highly suspect that these women didn’t go public decades ago, when Mr. Cosby could have used his considerable influence to sabotage any chance these women had at a career in show business and obliterate their credibility in media outlets that we

Include only URLs from the main `www.theonion.com` site, exclude any subdomains like `local.theonion.com`, `politics.theonion.com`, etc.

In [91]:
pattern = r'^https://www\.theonion\.com/'
sarcasm_df["is_main"] = sarcasm_df["url"].str.contains(pattern)

sarcasm_df.is_main.value_counts()

is_main
False    5913
True     5811
Name: count, dtype: int64

In [92]:
sarcasm_df = sarcasm_df[sarcasm_df.is_main].drop(columns=["is_main"])

len(sarcasm_df)

5811

In [93]:
sarcasm_df.url.values[0]

'https://www.theonion.com/cosby-lawyer-asks-why-accusers-didn-t-come-forward-to-b-1819577265'

Check if any of these articles are already present in `onion_df`:

In [97]:
onion_headlines = onion_df["headline"].str.lower().str.strip().values

sarcasm_df["headline"].isin(onion_headlines).sum()

3281

Exclude these to avoid duplication:

In [99]:
sarcasm_df = sarcasm_df[~sarcasm_df["headline"].isin(onion_headlines)]

len(sarcasm_df)

2530

Scrape the URLs:

In [102]:
sarcasm_df = sarcasm_df.reset_index(drop=True)

sarcasm_df.index

RangeIndex(start=0, stop=2530, step=1)

In [103]:
onion_div_class = 'entry-content single-post-content wp-block-post-content has-echo-font-size is-layout-flow wp-block-post-content-is-layout-flow'

sarcasm_df["body"] = None

In [104]:
sleep_times.extend([6, 7])

In [109]:
sarcasm_df.url.values[2]

'https://www.theonion.com/giant-altoid-heading-toward-earth-1819586323'

- NOTE: some of these articles are really old (the one above is from 1997) and the linked article doesn't contain any text.

In [111]:
for r in tqdm(sarcasm_df.itertuples()):
    if pd.isna(r.body):        
        request_headers = {
            'user-agent': random.choice(user_agents)
        }
        sarcasm_df.at[r.Index, "body"] = \
            scrape_website(r.url, onion_div_class, request_headers)

        time.sleep(random.choice(sleep_times))

0it [00:00, ?it/s]

No div with class 'entry-content single-post-content wp-block-post-content has-echo-font-size is-layout-flow wp-block-post-content-is-layout-flow' found.


2it [00:04,  2.07s/it]

No div with class 'entry-content single-post-content wp-block-post-content has-echo-font-size is-layout-flow wp-block-post-content-is-layout-flow' found.


3it [00:08,  2.92s/it]

...

2528it [3:37:35,  5.09s/it]

No div with class 'entry-content single-post-content wp-block-post-content has-echo-font-size is-layout-flow wp-block-post-content-is-layout-flow' found.


2529it [3:37:38,  4.22s/it]

No div with class 'entry-content single-post-content wp-block-post-content has-echo-font-size is-layout-flow wp-block-post-content-is-layout-flow' found.


2530it [3:37:41,  5.16s/it]


In [114]:
def format_time(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))

format_time(217 * 60 + 41.2)

'03:37:41'

- Took `03:37:41` to scrape `2,530` URLs on an 11th Gen i5-11300H with a random delay of upto 7 seconds between requests.

In [113]:
sarcasm_df.head()

,url,headline,body
0,https://www.theonion.com/cosby-lawyer-asks-why...,cosby lawyer asks why accusers didn't come for...,LOS ANGELES—Responding to recent allegations t...
1,https://www.theonion.com/after-careful-conside...,"after careful consideration, bush recommends o...",None
2,https://www.theonion.com/giant-altoid-heading-...,giant altoid heading toward earth,None
3,https://www.theonion.com/mobile-news-crew-repo...,mobile news crew reports on own van breaking down,None
4,https://www.theonion.com/north-dakota-not-hear...,north dakota not heard from in 48 hours,MIDWEST—Neighboring states are growing increas...


Exclude the articles that we couldn't fetch content for:

In [116]:
sarcasm_df.body.isna().value_counts(normalize=True)

body
True     0.556522
False    0.443478
Name: proportion, dtype: float64

- Over 55% of the URLs could not be scraped. This is a significant portion of the dataset.

In [118]:
sarcasm_df.to_csv("data/sarcasm_all.csv", index=False)

In [120]:
sarcasm_df = sarcasm_df[~sarcasm_df.body.isna()]

len(sarcasm_df)

1122

In [121]:
sarcasm_df.head()

,url,headline,body
0,https://www.theonion.com/cosby-lawyer-asks-why...,cosby lawyer asks why accusers didn't come for...,LOS ANGELES—Responding to recent allegations t...
4,https://www.theonion.com/north-dakota-not-hear...,north dakota not heard from in 48 hours,MIDWEST—Neighboring states are growing increas...
7,https://www.theonion.com/i-must-make-sure-you-...,'i must make sure you have the skills to pleas...,LONDON—Claiming that the fate of the Realm was...
13,https://www.theonion.com/bashar-al-assad-tries...,bashar al-assad tries tiny bit of sarin gas on...,DAMASCUS—Citing the ongoing international deba...
14,https://www.theonion.com/area-insurance-salesm...,area insurance salesman celebrates 14th year o...,"VALLEJO, CA—At a gala luncheon featuring Blood..."


In [122]:
sarcasm_df.to_csv("data/theonion2.csv", index=False)

# 2. Clickhole, Waterford Whisper News, The DailyER

https://huggingface.co/datasets/phosseini/multimodal_satire

- To obtain the full-text body of the articles, you need to scrape websites using the provided links in the dataset.

In [12]:
multi_ds = load_dataset("phosseini/multimodal_satire")

In [27]:
multi_df = pd.DataFrame(multi_ds['train'])

In [28]:
multi_df.head()

,url,headline,image_link,is_satire
0,https://babylonbee.com/news/this-aint-it-chief...,"'This Ain’t It, Chief' Updated To 'This Ain’t ...",https://babylonbee.com/img/articles/article-65...,1
1,https://babylonbee.com/news/announcement-the-b...,Using Our Prophetic Powers For Good: The Babyl...,https://babylonbee.com/img/articles/article-65...,1
2,https://babylonbee.com/news/study-wearing-a-ma...,Study Finds Connection Between Wearing A Mask ...,https://babylonbee.com/img/articles/article-65...,1
3,https://babylonbee.com/news/benny-hinn-flies-p...,Benny Hinn Travels To Dagobah To Master His Po...,https://babylonbee.com/img/articles/article-65...,1
4,https://babylonbee.com/news/white-house-adds-c...,White House Adds Cry Room For Fussy Reporters,https://babylonbee.com/img/articles/article-65...,1


In [29]:
multi_df = multi_df[multi_df.is_satire == 1].drop(columns=["is_satire"])

len(multi_df)

4000

In [30]:
multi_df['website'] = multi_df['url'].apply(lambda x: urlparse(x).netloc)

multi_df['website'].value_counts()

website
babylonbee.com               1000
clickhole.com                1000
waterfordwhispersnews.com    1000
thedailyer.com               1000
Name: count, dtype: int64

In [31]:
multi_df = multi_df[multi_df['website'] != "babylonbee.com"]

len(multi_df)

3000

In [32]:
multi_df.head()

,url,headline,image_link,website
1000,https://clickhole.com/youll-get-em-next-time-t...,You’ll Get ‘Em Next Time: This Woman Who’s Acc...,https://clickhole.com/wp-content/uploads/2020/...,clickhole.com
1001,https://clickhole.com/fascinating-chuck-e-chee...,Fascinating: Chuck E. Cheese Has Announced The...,https://clickhole.com/wp-content/uploads/2020/...,clickhole.com
1002,https://clickhole.com/neat-6-babies-that-we-ju...,Neat! 6 Babies That We Just Found Crawling Aro...,https://clickhole.com/wp-content/uploads/2020/...,clickhole.com
1003,https://clickhole.com/nature-is-incredible-sci...,Nature Is Incredible: Scientists Have Discover...,https://clickhole.com/wp-content/uploads/2020/...,clickhole.com
1004,https://clickhole.com/royal-heartbreak-queen-e...,Royal Heartbreak: Queen Elizabeth And Prince P...,https://clickhole.com/wp-content/uploads/2020/...,clickhole.com


In [96]:
test_urls = multi_df.groupby("website").head(1).url.values
test_urls

array(['https://clickhole.com/youll-get-em-next-time-this-woman-whos-accomplished-nothing-all-quarantine-figures-shell-knock-the-next-global-pandemic-out-of-the-park/',
       'https://waterfordwhispersnews.com/2020/06/29/health-authorities-warn-of-lethal-second-wave-of-fianna-fail/',
       'http://thedailyer.com/campus-rec-planing-locker-rooms-returned-normal-microbial-levels-time-fall-semester/'],
      dtype=object)

In [100]:
clickhole_test = scrape_website(test_urls[0], 'post-content')

In [101]:
print(clickhole_test)

If you haven’t yet found the motivation to make productive use of your time during quarantine, here’s a story that will inspire you not to give up hope: this woman who’s accomplished nothing all quarantine figures she’ll knock the next global pandemic out of the park.
Great outlook! You’ll get ‘em next time!
Although 29-year-old Rachel Vivano has spent the entirety of the COVID-19 pandemic sitting around binge-watching shows she’s already seen and scrolling on her phone, she’s not letting it get her down. Instead, she’s opted to take it easy for the time being, feeling reasonably confident that next time all of human civilization is brought to its knees by an unrelenting contagion of biblical scale, she’ll be mentally prepared to go all-in on enriching activities like learning to bake bread, getting into gardening, and completely reorganizing her apartment.
“Look, I’ll admit that I definitely haven’t made the best use of my quarantine time, but I figure this can sort of just be a pract

In [82]:
waterford_test = scrape_website(test_urls[1])

waterford_test

Error fetching https://waterfordwhispersnews.com/2020/06/29/health-authorities-warn-of-lethal-second-wave-of-fianna-fail/: 403 Client Error: Forbidden for url: https://waterfordwhispersnews.com/2020/06/29/health-authorities-warn-of-lethal-second-wave-of-fianna-fail/


In [83]:
dailyer_test = scrape_website(test_urls[2])

dailyer_test

Error fetching http://thedailyer.com/campus-rec-planing-locker-rooms-returned-normal-microbial-levels-time-fall-semester/: 406 Client Error: Not Acceptable for url: http://thedailyer.com/campus-rec-planing-locker-rooms-returned-normal-microbial-levels-time-fall-semester/


## Scrape Clickhole articles

- 1000 articles

In [34]:
clickhole_df = multi_df[multi_df['website'] == "clickhole.com"]

len(clickhole_df)

1000

In [ ]:
clickhole_df['body'] = None

In [56]:
for r in tqdm(clickhole_df.itertuples()):
    if pd.isna(r.body):        
        request_headers = {
            'user-agent': random.choice(user_agents)
        }
        clickhole_df.at[r.Index, "body"] = \
            scrape_website(r.url, 'post-content', request_headers)

        time.sleep(random.choice(sleep_times))

1000it [46:17,  2.78s/it]


- Took `46:17` to scrape `1,000` URLs on an 11th Gen i5-11300H with a random delay of upto 5 seconds between requests.

In [57]:
clickhole_df.head()

,url,headline,image_link,website,body
0,https://clickhole.com/youll-get-em-next-time-t...,You’ll Get ‘Em Next Time: This Woman Who’s Acc...,https://clickhole.com/wp-content/uploads/2020/...,clickhole.com,If you haven’t yet found the motivation to mak...
1,https://clickhole.com/fascinating-chuck-e-chee...,Fascinating: Chuck E. Cheese Has Announced The...,https://clickhole.com/wp-content/uploads/2020/...,clickhole.com,Just weeks after filing for Chapter 11 bankrup...
2,https://clickhole.com/neat-6-babies-that-we-ju...,Neat! 6 Babies That We Just Found Crawling Aro...,https://clickhole.com/wp-content/uploads/2020/...,clickhole.com,"Exciting news, everyone! We were just taking a..."
3,https://clickhole.com/nature-is-incredible-sci...,Nature Is Incredible: Scientists Have Discover...,https://clickhole.com/wp-content/uploads/2020/...,clickhole.com,"Nature is a remarkable place, and scientists a..."
4,https://clickhole.com/royal-heartbreak-queen-e...,Royal Heartbreak: Queen Elizabeth And Prince P...,https://clickhole.com/wp-content/uploads/2020/...,clickhole.com,The United Kingdom is in mourning today after ...


In [58]:
clickhole_df.body.isna().sum()

0

In [61]:
clickhole_df["body"].sample(10).values

array(['',
       'If you’ve ever let fear or uncertainty stop you from going out on a limb and trying something new, it was likely for the best, as here’s a story that proves that good things rarely come from diverging from routine: This guy briefly toyed around with some of the different settings on his shower head only to immediately come crawling back to the normal one after realizing the others were all way shittier.\nIf it ain’t broke, don’t fix it. Idiot.\nWhen Max Kenlock of Des Moines, Iowa, turned on the water for his post-shower workout this morning, he was suddenly possessed with the pompous notion that he could somehow do better than the good ol’ standard setting on the chrome shower head he bought for $21 on Amazon last month. But it wasn’t long before his hubris came back to bite him in the ass, as he quickly realized that the other settings—with pleasant-sounding names like “RAIN,” “MASSAGE JET,” and “MORNING MIST”— did not even come close to delivering the heavenly, sp

In [62]:
clickhole_df.to_csv("data/clickhole.csv", index=False)

# Merge all the datasets

In [123]:
onion1 = pd.read_csv("data/theonion.csv")
onion1["website"] = "The Onion"

onion2 = pd.read_csv("data/theonion2.csv")
onion2["website"] = "The Onion"

clickhole = pd.read_csv("data/clickhole.csv")
clickhole["website"] = "Clickhole"

In [124]:
data = pd.concat([onion1, onion2, clickhole], ignore_index=True)

len(data)

36002

In [125]:
data.head()

,headline,body,website,url,image_link
0,Relaxed Marie Kondo Now Says She Perfectly Hap...,LOS ANGELES—Admitting that she’d made some maj...,The Onion,NaN,NaN
1,U.S. Officials Call For Correct Amount Of Viol...,WASHINGTON—Addressing the need for swift and m...,The Onion,NaN,NaN
2,Kamala Harris Asks Communications Assistant If...,WASHINGTON—In an effort to expand her professi...,The Onion,NaN,NaN
3,25 Arrested In Fake Nursing School Diploma Scheme,Federal agents have arrested 25 suspects accus...,The Onion,NaN,NaN
4,World’s Oldest American Dies At 72,"OXNARD, CA—Living longer than any citizen in t...",The Onion,NaN,NaN


In [126]:
data.to_csv("data/satirical_news_articles.csv", index=False)

# Dataset Statistics